# Extra task
<b>This task is not graded, you are not required to submit this. This is just for your learning only attempt this if you have extra time</b>

# POS Tagging

For this task we will be using the many to many archetecture(the first one from the left)
<img src="archetecturernn.png">
This archtecture is called the encoder-decoder or the sequence 2 sequence(Seq2seq) model. For this task we will be using the brown [Brown Corpus](https://en.wikipedia.org/wiki/Brown_Corpus), you can import this from nltk.corpus.Here is a visual guide for how this will work:
![alt text](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)
This image ilustrates the task of translation from one language to another. We will instead use the same techniques for POS tagging.<br>
<b>How the mode works:</b> The model uses 2 LSTM's. The first LSTM encodes the sentence into feature vector of sorts and the second LSTM then uses this vector to predict a sequence of POS tags. After each prediction we will provide the actual tag instead of the predicted so that our model learns faster this technique is called <a href="https://towardsdatascience.com/what-is-teacher-forcing-3da6217fed1c">Teacher Forcing</a><br>


<h3>Data Preperation</h3>
<ul>
    <li> First we need to preprocess the data, convert the data to lower casing.</li>
    <li> Split the data into test, train and validation in the ratio 20,70,10. Use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">scikit_test_train_split</a> <br><i><b>Hint:</b> use the splitter twice to get desired data splits.</i></li>
    <li> For this task our model will have 2 inputs the sentence and the actual tag sequence. The labels/targets will also be the actual tag sequence.Inputs=(sentence,input tag sequence), Output=(output tag sequence)</li>
    <li> For the input tag sequence append start of sentence '[START]'or '[SOS] tag at start of the sentence and for prediction tag sequence append  end of sentence '[STOP]' or '[EOS]' tag. These tag help the model learn when to start and end a sequence of predictions</li>
    <li> Repeat the process of extracting vocabulary and numeric encoding for both the inputs and one hot encodings for prediction tag sequence.<li>
</ul>
<h3>Embeddings</h3>
<ul>
    <li>For this part instead of using pretrained embeddings you will use train them from scratch</li>
</ul>
<h3>Create Model</h3>
<ul>
    <li> Create the model using <a href"https://www.tensorflow.org/guide/keras/functional">functional API</a></li>
    <li> Hints: The emebedding layer has a parameter that allows you to use pretrained embeddings, for shared layers read the section of shared layer weights in function API docs</li>

In [2]:
%matplotlib inline
import numpy as np
from IPython.display import Image
# Get the interactive Tools for Matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd
from nltk import ngrams
import pandas as pd
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, RepeatVector,Flatten, TimeDistributed, Input,Bidirectional,LocallyConnected1D,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.layers import Embedding, LSTM ,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
# from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
# import tensorflow.keras.utils.to_categorical as to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.model_selection import train_test_split as splitter
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import math
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('brown')
from nltk.corpus import brown
from gensim.models import Word2Vec

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [0]:
def preprocessing(data):
  sentences = []
  sen=[]
  for i in data:
    for j in i:
      z=re.match('[a-zA-Z\-]+',j)
      if z:
        j=j.lower()
      sen.append(j)
    sentences.append(sen)
    sen=[]
  return sentences

data = preprocessing(brown.sents())
tags = list(brown.tagged_sents())

for i in range(0,len(tags)):
  tags[i]=[j[1] for j in tags[i]]

In [0]:
#SOS EOS
tags2=[] 
for i in tags:
  tags2.append( ['SOS'] + i + ['EOS'])


In [0]:
tokenizer2=Tokenizer()
tokenizer2.fit_on_texts(tags2)


In [0]:
def testTrainSplit(data_X,data_Y):

    train_data,valid_data,train_labels,valid_labels=splitter(data_X,data_Y,train_size=0.9,test_size=0.1,shuffle=True)
    train_data,test_data,train_labels,test_labels=splitter(train_data,train_labels,train_size=0.8,test_size=0.2,shuffle=True)

    return train_data,train_labels,test_data,test_labels,valid_data,valid_labels

train_data,train_labels,test_data,test_labels,valid_data,valid_labels=testTrainSplit(data,tags)



In [0]:
tokenizer1=Tokenizer(split=' ',oov_token="UNK")
tokenizer.fit_on_texts(train_data)


In [0]:
#generating 
train_vocab = list(tokenizer.word_index.keys())
tag_vocab = list(tokenizer2.word_index.keys())

In [0]:
def label_seq(data):
  decoder_in_seq=[]
  decoder_out_seq=[]
  for i in data:
    decoder_in_seq.append(['SOS'] + i)
  for i in data:
    decoder_out_seq.append(i + ['EOS'])
  return decoder_in_seq,decoder_out_seq


In [0]:
train_decoder_in,train_decoder_out=label_seq(train_labels)
test_decoder_in,test_decoder_out=label_seq(test_data)
valid_decoder_in,valid_decoder_out=label_seq(valid_data)

In [0]:
def text2seq(tokenizer,data):
  seq = tokenizer.texts_to_sequences(data)
  seq=pad_sequences(seq,padding="post",maxlen=15)
  return seq

train_seq=text2seq(tokenizer,train_data)
test_seq=text2seq(tokenizer,test_data)
valid_seq=text2seq(tokenizer,valid_data)

train_decoder_in=text2seq(tokenizer2,train_decoder_in)
test_decoder_in=text2seq(tokenizer2,test_decoder_in)
valid_decoder_in=text2seq(tokenizer2,valid_decoder_in)

train_decoder_out=text2seq(tokenizer2,train_decoder_out)
test_decoder_out=text2seq(tokenizer2,test_decoder_out)
valid_decoder_out=text2seq(tokenizer2,valid_decoder_out)




In [131]:
train_decoder_in[0]

array([19,  4,  1, 10, 19,  4,  1, 21, 11, 15,  3,  4, 18,  1,  5],
      dtype=int32)

In [132]:
#model 
encoder_inputs = Input(shape=(15,))
emb1 = Embedding(len(train_vocab),100,input_length=15,trainable=True)(encoder_inputs)
lstm_encoder, state_h, state_c=LSTM(100, return_sequences=True, return_state=True,recurrent_dropout=0.2)(emb1)#apply bidirectional lstm
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(15,))
emb2 = Embedding(len(tag_vocab),100,input_length=15,trainable=True)(decoder_inputs)
lstm_decoder,_,_= LSTM(100,return_sequences=True, return_state=True,recurrent_dropout=0.2)(emb2,initial_state=[state_h, state_c])#apply bidirectional lstm
decoder_dense = Dense(len(tag_vocab), activation='softmax')(lstm_decoder)

model = Model(inputs=[encoder_inputs,decoder_inputs], outputs=[decoder_dense])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_crossentropy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 15)]         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 15, 100)      4981600     input_11[0][0]                   
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 15, 100)      47400       input_12[0][0]                   
____________________________________________________________________________________________

In [133]:
model.fit([np.array(train_seq),np.array(train_decoder_in)],train_labels, epochs=20, batch_size=32,verbose=1,shuffle=True)

ValueError: ignored